# Análisis del conjunto de datos "Flor Iris" mediante un modelo de red neuronal MLPC

![](https://www.saedsayad.com/images/Perceptron_bkp_1.png)

## Aproximación mediante el método de agrupamiento k-Means

* ROJO   ->   Setosa
* AZUL   ->   Versicolor
* VERDE  ->   Virginica

![](kmeans_fallido.png)

Se puede comprobar que el cálculo de los centroides no es muy efectivo para este conjunto de datos ya que se pueden observar agrupaciones de datos muy dispersos

## Aproximación mediante MLPC

### Importación de dependencias

In [36]:
import $ivy.`org.apache.spark::spark-sql:2.4.4`
import $ivy.`org.apache.spark::spark-mllib:2.4.4`
import org.apache.spark.sql._
import org.apache.log4j.{Level, Logger}
import java.sql.Timestamp
import org.apache.spark.sql.{Dataset, SparkSession}
import org.apache.spark.sql.types._
import org.apache.spark.ml.feature._

Logger.getLogger("org").setLevel(Level.OFF)

import $ivy.$                                  

import $ivy.$                                    

import org.apache.spark.sql._

import org.apache.log4j.{Level, Logger}

import java.sql.Timestamp

import org.apache.spark.sql.{Dataset, SparkSession}

import org.apache.spark.sql.types._

import org.apache.spark.ml.feature._



### Sesión de spark en el entorno del notebook

In [37]:
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

Getting spark JARs
Creating SparkSession


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@5f6165a4

### Estructura para el almacenamiento de los datos de entrada

In [38]:
val schema = StructType(
      StructField("Longitud_Sepalo", DoubleType, nullable = false) ::
        StructField("Anchura_Sepalo", DoubleType, nullable = true) ::
        StructField("Longitud_Petalo", DoubleType, nullable = true) ::
        StructField("Anchura_Petalo", DoubleType, nullable = true) ::
        StructField("Label", StringType, nullable = true) :: Nil)

schema: StructType = StructType(
  StructField("Longitud_Sepalo", DoubleType, false, {}),
  StructField("Anchura_Sepalo", DoubleType, true, {}),
  StructField("Longitud_Petalo", DoubleType, true, {}),
  StructField("Anchura_Petalo", DoubleType, true, {}),
  StructField("Label", StringType, true, {})
)

### Capturando los datos de entrada

In [39]:
val csvData = spark.read.options(Map(
    "header" -> "true",
    "ignoreLeadingWhiteSpace" -> "true",
    "ignoreTrailingWhiteSpace" -> "true",
    "timestampFormat" -> "MM/dd/yyyy HH:mm:ss a",
    "mode" -> "DROPMALFORMED"))
    .schema(schema)
    .csv("C:/sw/iris.csv")

csvData: DataFrame = [Longitud_Sepalo: double, Anchura_Sepalo: double ... 3 more fields]

### Muestra de los datos obtenidos

In [40]:
csvData.show(5)

show at cmd39.sc:1

1 / 1

show at cmd39.sc:1

1 / 1

show at cmd39.sc:1

1 / 1

+---------------+--------------+---------------+--------------+------+
|Longitud_Sepalo|Anchura_Sepalo|Longitud_Petalo|Anchura_Petalo| Label|
+---------------+--------------+---------------+--------------+------+
|            5.1|           3.5|            1.4|           0.2|setosa|
|            4.9|           3.0|            1.4|           0.2|setosa|
|            4.7|           3.2|            1.3|           0.2|setosa|
|            4.6|           3.1|            1.5|           0.2|setosa|
|            5.0|           3.6|            1.4|           0.2|setosa|
+---------------+--------------+---------------+--------------+------+
only showing top 5 rows



### Se crea una columna que agrupa las características en forma de vector

In [41]:
val assembler = new VectorAssembler()
  .setInputCols(Array("Longitud_Sepalo", "Anchura_Sepalo", "Longitud_Petalo", "Anchura_Petalo"))
  .setOutputCol("Features")

val preFitData = assembler.transform(csvData)
features.show(5)

show at cmd40.sc:6

1 / 1

show at cmd40.sc:6

1 / 1

show at cmd40.sc:6

1 / 1

+---------------+--------------+---------------+--------------+------+-----------------+
|Longitud_Sepalo|Anchura_Sepalo|Longitud_Petalo|Anchura_Petalo| Label|         Features|
+---------------+--------------+---------------+--------------+------+-----------------+
|            5.1|           3.5|            1.4|           0.2|setosa|[5.1,3.5,1.4,0.2]|
|            4.9|           3.0|            1.4|           0.2|setosa|[4.9,3.0,1.4,0.2]|
|            4.7|           3.2|            1.3|           0.2|setosa|[4.7,3.2,1.3,0.2]|
|            4.6|           3.1|            1.5|           0.2|setosa|[4.6,3.1,1.5,0.2]|
|            5.0|           3.6|            1.4|           0.2|setosa|[5.0,3.6,1.4,0.2]|
+---------------+--------------+---------------+--------------+------+-----------------+
only showing top 5 rows



assembler: VectorAssembler = vecAssembler_051b3d42c506
preFitData: DataFrame = [Longitud_Sepalo: double, Anchura_Sepalo: double ... 4 more fields]

In [42]:
val labelIndexer = new StringIndexer().setInputCol("Label").setOutputCol("LabelIndexado").fit(preFitData)
val etiquetas = labelIndexer.labels.mkString("[", ", ", "]")
println(s"Tipos de labels encontrados: $etiquetas")

countByValue at StringIndexer.scala:140

1 / 1

countByValue at StringIndexer.scala:140

1 / 1

countByValue at StringIndexer.scala:140

...

1 / 1

countByValue at StringIndexer.scala:140

1 / 1

Tipos de labels encontrados: [versicolor, virginica, setosa]


labelIndexer: StringIndexerModel = strIdx_05345d82852f
etiquetas: String = "[versicolor, virginica, setosa]"

In [43]:
val featureIndexer = new VectorIndexer().setInputCol("Features").setOutputCol("FeaturesIndexadas").setMaxCategories(4).fit(preFitData)

take at VectorIndexer.scala:145

1 / 1

take at VectorIndexer.scala:145

1 / 1

take at VectorIndexer.scala:145

...

reduce at VectorIndexer.scala:154

1 / 1

reduce at VectorIndexer.scala:154

1 / 1

featureIndexer: VectorIndexerModel = vecIdx_42fdaa79e3dd

### Reparto de registros para entrenamiento y pruebas

In [44]:
val splits = preFitData.randomSplit(Array(0.6, 0.4))
val trainingData = splits(0)
val testData = splits(1)

splits: Array[Dataset[Row]] = Array(
  [Longitud_Sepalo: double, Anchura_Sepalo: double ... 4 more fields],
  [Longitud_Sepalo: double, Anchura_Sepalo: double ... 4 more fields]
)
trainingData: Dataset[Row] = [Longitud_Sepalo: double, Anchura_Sepalo: double ... 4 more fields]
testData: Dataset[Row] = [Longitud_Sepalo: double, Anchura_Sepalo: double ... 4 more fields]

### Estructura representativa de una red neuronal multicapa

![](https://dsc-spidal.github.io/harp/img/nn.png)

In [45]:
//Definición de capas que componen la red neuronal
val percentronsCapaEntrada = 4
val percentronsCapaIntermedia = 5
val percentronsCapaSalida = 3
val layers = Array[Int](percentronsCapaEntrada, percentronsCapaIntermedia, percentronsCapaIntermedia, percentronsCapaSalida)

percentronsCapaEntrada: Int = 4
percentronsCapaIntermedia: Int = 5
percentronsCapaSalida: Int = 3
layers: Array[Int] = Array(4, 5, 5, 3)

### Se configura el entrenador

In [46]:
import org.apache.spark.ml.classification.MultilayerPerceptronClassifier

// create the trainer and set its parameters
val trainer = new MultilayerPerceptronClassifier()
  .setLayers(layers)
  .setLabelCol("LabelIndexado")
  .setFeaturesCol("FeaturesIndexadas")
  .setBlockSize(128)
  .setSeed(System.currentTimeMillis)
  .setMaxIter(200)

import org.apache.spark.ml.classification.MultilayerPerceptronClassifier

// create the trainer and set its parameters

trainer: MultilayerPerceptronClassifier = mlpc_c15e9f68b8ac

In [47]:
import org.apache.spark.ml.feature.IndexToString

// Convert indexed labels back to original labels.
val labelConverter = new IndexToString()
  .setInputCol("prediction")
  .setOutputCol("LabelPredicho")
  .setLabels(labelIndexer.labels)

import org.apache.spark.ml.feature.IndexToString

// Convert indexed labels back to original labels.

labelConverter: IndexToString = idxToStr_2605157052f6

In [48]:
import org.apache.spark.ml.Pipeline

// Chain indexers and MultilayerPerceptronClassifier in a Pipeline.
val pipeline = new Pipeline()
  .setStages(Array(labelIndexer, featureIndexer, trainer, labelConverter))

import org.apache.spark.ml.Pipeline

// Chain indexers and MultilayerPerceptronClassifier in a Pipeline.

pipeline: Pipeline = pipeline_5a960d9989df

In [ ]:
// Train model. This also runs the indexers.
val model = pipeline.fit(trainingData)

count at LBFGS.scala:195

1 / 1

count at LBFGS.scala:195

1 / 1

count at LBFGS.scala:195

...

1 / 1

treeAggregate at LBFGS.scala:261

1 / 1

treeAggregate at LBFGS.scala:261

1 / 1

In [ ]:
// Make predictions.
val predictions = model.transform(testData)

In [ ]:
// Select example rows to display.
predictions.show(5)

In [ ]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

// Select (prediction, true label) and compute test error.
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("LabelIndexado")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
val accuracy = evaluator.evaluate(predictions)
println("Test Error = " + (1.0 - accuracy))

In [ ]:
import org.apache.spark.ml.classification.MultilayerPerceptronClassificationModel

val mlpc = model.stages(2).asInstanceOf[MultilayerPerceptronClassificationModel]

println(s"Modelo de clasificación:\n$mlpc")
println(s"Parámetros: ${mlpc.explainParams}")
println(s"Pesos: ${mlpc.weights}")